In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 10: Monitoring Station ---</h2><p>You fly into the asteroid belt and reach the Ceres monitoring station.  The Elves here have an emergency: they're having trouble tracking all of the asteroids and can't be sure they're safe.</p>
<p>The Elves would like to build a new monitoring station in a nearby area of space; they hand you a map of all of the asteroids in that region (your puzzle input).</p>
<p>The map indicates whether each position is empty (<code>.</code>) or contains an asteroid (<code>#</code>).  The asteroids are much smaller than they appear on the map, and every asteroid is exactly in the center of its marked position.  The asteroids can be described with <code>X,Y</code> coordinates where <code>X</code> is the distance from the left edge and <code>Y</code> is the distance from the top edge (so the top-left corner is <code>0,0</code> and the position immediately to its right is <code>1,0</code>).</p>
<p>Your job is to figure out which asteroid would be the best place to build a <em>new monitoring station</em>. A monitoring station can <em>detect</em> any asteroid to which it has <em>direct line of sight</em> - that is, there cannot be another asteroid <em>exactly</em> between them. This line of sight can be at any angle, not just lines aligned to the grid or <span title="The Elves on Ceres are clearly not concerned with honor.">diagonally</span>. The <em>best</em> location is the asteroid that can <em>detect</em> the largest number of other asteroids.</p>
<p>For example, consider the following map:</p>
<pre><code>.#..#
.....
#####
....#
...<em>#</em>#
</code></pre>
<p>The best location for a new monitoring station on this map is the highlighted asteroid at <code>3,4</code> because it can detect <code>8</code> asteroids, more than any other location. (The only asteroid it cannot detect is the one at <code>1,0</code>; its view of this asteroid is blocked by the asteroid at <code>2,2</code>.) All other asteroids are worse locations; they can detect <code>7</code> or fewer other asteroids. Here is the number of other asteroids a monitoring station on each asteroid could detect:</p>
<pre><code>.7..7
.....
67775
....7
...87
</code></pre>
<p>Here is an asteroid (<code>#</code>) and some examples of the ways its line of sight might be blocked. If there were another asteroid at the location of a capital letter, the locations marked with the corresponding lowercase letter would be blocked and could not be detected:</p>
<pre><code>#.........
...A......
...B..a...
.EDCG....a
..F.c.b...
.....c....
..efd.c.gb
.......c..
....f...c.
...e..d..c
</code></pre>
<p>Here are some larger examples:</p>
<ul>
<li><p>Best is <code>5,8</code> with <code>33</code> other asteroids detected:</p>
<pre><code>......#.#.
#..#.#....
..#######.
.#.#.###..
.#..#.....
..#....#.#
#..#....#.
.##.#..###
##...<em>#</em>..#.
.#....####
</code></pre></li>
<li><p>Best is <code>1,2</code> with <code>35</code> other asteroids detected:</p>
<pre><code>#.#...#.#.
.###....#.
.<em>#</em>....#...
##.#.#.#.#
....#.#.#.
.##..###.#
..#...##..
..##....##
......#...
.####.###.
</code></pre></li>
<li><p>Best is <code>6,3</code> with <code>41</code> other asteroids detected:</p>
<pre><code>.#..#..###
####.###.#
....###.#.
..###.<em>#</em>#.#
##.##.#.#.
....###..#
..#.#..#.#
#..#.#.###
.##...##.#
.....#.#..
</code></pre></li>
<li><p>Best is <code>11,13</code> with <code>210</code> other asteroids detected:</p>
<pre><code>.#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.####<em>#</em>#####...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##
</code></pre></li>
</ul>
<p>Find the best location for a new monitoring station.  <em>How many other asteroids can be detected from that location?</em></p>
</article>


In [138]:
from math import atan2, pi, sqrt
from operator import length_hint

from more_itertools import first, tail


tests = [
    {
        "name": "Example 1",
        "map": (
            """
                .#..#
                .....
                #####
                ....#
                ...##
            """
        ),
        "expected": (3, 4, 8),
    },
    {
        "name": "Example 2",
        "map": (
            """
                ......#.#.
                #..#.#....
                ..#######.
                .#.#.###..
                .#..#.....
                ..#....#.#
                #..#....#.
                .##.#..###
                ##...#..#.
                .#....####
            """
        ),
        "expected": (5, 8, 33),
    },
    {
        "name": "Example 3",
        "map": (
            """
                #.#...#.#.
                .###....#.
                .#....#...
                ##.#.#.#.#
                ....#.#.#.
                .##..###.#
                ..#...##..
                ..##....##
                ......#...
                .####.###.
            """
        ),
        "expected": (1, 2, 35),
    },
    {
        "name": "Example 4",
        "map": (
            """
                .#..#..###
                ####.###.#
                ....###.#.
                ..###.##.#
                ##.##.#.#.
                ....###..#
                ..#.#..#.#
                #..#.#.###
                .##...##.#
                .....#.#..
            """
        ),
        "expected": (6, 3, 41),
    },
    {
        "name": "Example 5",
        "map": (
            """
                .#..##.###...#######
                ##.############..##.
                .#.######.########.#
                .###.#######.####.#.
                #####.##.#.##.###.##
                ..#####..#.#########
                ####################
                #.####....###.#.#.##
                ##.#################
                #####.##.###..####..
                ..######..##.#######
                ####.##.####...##..#
                .#####..#.######.###
                ##...#.##########...
                #.##########.#######
                .####.#.###.###.#.##
                ....##.##.###..#####
                .#.#.###########.###
                #.#.#.#####.####.###
                ###.##.####.##.#..##
            """
        ),
        "expected": (11, 13, 210),
    },
]


def magnitude(x1: int, y1: int, x2: int, y2: int) -> float:
    return sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)


def angle(x1: int, y1: int, x2: int, y2: int) -> float:
    return atan2(y1 - y2, x1 - x2)


class Map:
    def __init__(self, map: str) -> None:
        self.map = [l.strip() for l in map.strip().splitlines()]

        self.xs, self.ys = len(self.map[0]), len(self.map)

        self.astroids = {
            (x, y): 0
            for x, y in product(range(self.xs), range(self.ys))
            if self.map[y][x] == "#"
        }

    def ray_trace(self):
        for a1 in self.astroids:
            angles = {}
            for a2 in self.astroids:
                if a1 != a2:
                    ang = angle(*a1, *a2)
                    mag = magnitude(*a1, *a2)
                    if mag < angles.get(ang, inf):
                        angles[ang] = mag

            self.astroids[a1] += len(angles)

    def ray_trace_single(self, a1, a2):
        ang = angle(*a1, *a2)
        mag = magnitude(*a1, *a2)
        point = a2
        for aa in self.astroids:
            if a1 != aa:
                ang1 = angle(*a1, *aa)
                mag1 = magnitude(*a1, *aa)
                if mag1 < mag and ang1 == ang:
                    point = aa
        return point == a2

    def see_most_astroids(self) -> tuple[int, int, int]:
        return max(((*k, v) for k, v in self.astroids.items()), key=lambda t: t[-1])

    def __repr__(self) -> str:
        return "\n".join(
            "".join(
                c if c == "." else str(self.astroids[(x, y)]) for x, c in enumerate(l)
            )
            for y, l in enumerate(self.map)
        )


def test_best_location(map: str) -> int:
    m = Map(map)
    m.ray_trace()
    return m.see_most_astroids()


def test_line_of_sight():
    example = """
    #.........
    ...#......
    ...#..#...
    .####....#
    ..#.#.#...
    .....#....
    ..###.#.##
    .......#..
    ....#...#.
    ...#..#..#
    """
    m = Map(example)
    #   0123456789
    # 0 #.........
    # 1 ...A......
    # 2 ...B..a...
    # 3 .EDCG....a
    # 4 ..F.c.b...
    # 5 .....c....
    # 6 ..efd.c.gb
    # 7 .......c..
    # 8 ....f...c.
    # 9 ...e..d..c
    #   0123456789

    o = 0, 0
    A = 3, 1
    B = 3, 2
    C = 3, 3
    D = 2, 3
    E = 1, 3
    F = 2, 4
    G = 4, 3
    ax = (6, 2), (9, 3)
    bs = (6, 4), (9, 6)
    cs = (4, 4), (5, 5), (6, 6), (7, 7), (8, 8), (9, 9)
    ds = (4, 6), (6, 9)
    es = (2, 6), (3, 9)
    fs = (3, 6), (4, 8)
    gs = ((8, 6),)

    def in_line_of_sight(a1, a2):
        return m.ray_trace_single(a1, a2)

    assert in_line_of_sight(o, A)
    assert in_line_of_sight(o, B)
    assert in_line_of_sight(o, C)
    assert in_line_of_sight(o, D)
    assert in_line_of_sight(o, E)
    assert in_line_of_sight(o, F)
    assert in_line_of_sight(o, G)

    assert not any(in_line_of_sight(o, a) for a in ax)
    assert not any(in_line_of_sight(o, a) for a in bs)
    assert not any(in_line_of_sight(o, a) for a in cs)
    assert not any(in_line_of_sight(o, a) for a in ds)
    assert not any(in_line_of_sight(o, a) for a in es)
    assert not any(in_line_of_sight(o, a) for a in fs)
    assert not any(in_line_of_sight(o, a) for a in gs)


test_line_of_sight()


run_tests_params(test_best_location, tests)


Test Example 1 passed, for test_best_location.
Test Example 2 passed, for test_best_location.
Test Example 3 passed, for test_best_location.
Test Example 4 passed, for test_best_location.
Test Example 5 passed, for test_best_location.
Success


In [141]:
with open("../input/day10.txt") as f:
    puzzle = f.read()

print(f"Part I: {test_best_location(puzzle)}")

Part I: (22, 25, 286)


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>286</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Once you give them the coordinates, the Elves quickly deploy an Instant Monitoring Station to the location and discover <span title="The Elves on Ceres just have a unique system of values, that's all.">the worst</span>: there are simply too many asteroids.</p>
<p>The only solution is <em>complete vaporization by giant laser</em>.</p>
<p>Fortunately, in addition to an asteroid scanner, the new monitoring station also comes equipped with a giant rotating laser perfect for vaporizing asteroids. The laser starts by pointing <em>up</em> and always rotates <em>clockwise</em>, vaporizing any asteroid it hits.</p>
<p>If multiple asteroids are <em>exactly</em> in line with the station, the laser only has enough power to vaporize <em>one</em> of them before continuing its rotation. In other words, the same asteroids that can be <em>detected</em> can be vaporized, but if vaporizing one asteroid makes another one detectable, the newly-detected asteroid won't be vaporized until the laser has returned to the same position by rotating a full 360 degrees.</p>
<p>For example, consider the following map, where the asteroid with the new monitoring station (and laser) is marked <code>X</code>:</p>
<pre><code>.#....#####...#..
##...##.#####..##
##...#...#.#####.
..#.....X...###..
..#.#.....#....##
</code></pre>
<p>The first nine asteroids to get vaporized, in order, would be:</p>
<pre><code>.#....###<em>2</em><em>4</em>...#..
##...##.<em>1</em><em>3</em>#<em>6</em><em>7</em>..<em>9</em>#
##...#...<em>5</em>.<em>8</em>####.
..#.....X...###..
..#.#.....#....##
</code></pre>
<p>Note that some asteroids (the ones behind the asteroids marked <code>1</code>, <code>5</code>, and <code>7</code>) won't have a chance to be vaporized until the next full rotation.  The laser continues rotating; the next nine to be vaporized are:</p>
<pre><code>.#....###.....#..
##...##...#.....#
##...#......<em>1</em><em>2</em><em>3</em><em>4</em>.
..#.....X...<em>5</em>##..
..#.<em>9</em>.....<em>8</em>....<em>7</em><em>6</em>
</code></pre>
<p>The next nine to be vaporized are then:</p>
<pre><code>.<em>8</em>....###.....#..
<em>5</em><em>6</em>...<em>9</em>#...#.....#
<em>3</em><em>4</em>...<em>7</em>...........
..<em>2</em>.....X....##..
..<em>1</em>..............
</code></pre>
<p>Finally, the laser completes its first full rotation (<code>1</code> through <code>3</code>), a second rotation (<code>4</code> through <code>8</code>), and vaporizes the last asteroid (<code>9</code>) partway through its third rotation:</p>
<pre><code>......<em>2</em><em>3</em><em>4</em>.....<em>6</em>..
......<em>1</em>...<em>5</em>.....<em>7</em>
.................
........X....<em>8</em><em>9</em>..
.................
</code></pre>
<p>In the large example above (the one with the best monitoring station location at <code>11,13</code>):</p>
<ul>
<li>The 1st asteroid to be vaporized is at <code>11,12</code>.</li>
<li>The 2nd asteroid to be vaporized is at <code>12,1</code>.</li>
<li>The 3rd asteroid to be vaporized is at <code>12,2</code>.</li>
<li>The 10th asteroid to be vaporized is at <code>12,8</code>.</li>
<li>The 20th asteroid to be vaporized is at <code>16,0</code>.</li>
<li>The 50th asteroid to be vaporized is at <code>16,9</code>.</li>
<li>The 100th asteroid to be vaporized is at <code>10,16</code>.</li>
<li>The 199th asteroid to be vaporized is at <code>9,6</code>.</li>
<li><em>The 200th asteroid to be vaporized is at <code>8,2</code>.</em></li>
<li>The 201st asteroid to be vaporized is at <code>10,9</code>.</li>
<li>The 299th and final asteroid to be vaporized is at <code>11,1</code>.</li>
</ul>
<p>The Elves are placing bets on which will be the <em>200th</em> asteroid to be vaporized.  Win the bet by determining which asteroid that will be; <em>what do you get if you multiply its X coordinate by <code>100</code> and then add its Y coordinate?</em> (For example, <code>8,2</code> becomes <em><code>802</code></em>.)</p>
</article>

</main>


In [4]:
# TODO: https://twitter.com/driscollis/status/1802748074114322484

In [5]:
# print(f"Part II: {first_frequency_encounterd_twice(puzzle)}")

<link href="style.css" rel="stylesheet"></link>
...
